In [36]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [37]:
#import dataframe
df = pd.read_excel('marketing_data2.xlsx')
df.head()

,ID,Year_Birth,Education,Marital_Status2,Income2,Dt_Customer2,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,NumWebPurchases,NumWebVisitsMonth,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,MntTotal
0,1826,1970,Graduation,Divorced,84835.0,2014-06-16,0,189,104,379,111,4,1,0,0,0,0,0,783
1,1,1961,Graduation,Single,57091.0,2014-06-15,0,464,5,64,7,7,5,0,1,0,0,0,540
2,10476,1958,Graduation,Married,67267.0,2014-05-13,0,134,11,59,15,3,2,0,0,0,0,0,219
3,1386,1967,Graduation,Together,32474.0,2014-05-11,0,10,0,1,0,1,7,0,0,0,0,0,11
4,5371,1989,Graduation,Single,21474.0,2014-04-08,0,6,16,24,11,3,7,0,0,1,0,0,57


In [38]:
df = df.drop_duplicates()

In [39]:
#create copy for edited dataframe
rfm = df.copy()

#drop several 'Mnt' columns
rfm = rfm.drop(columns=['MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts'])

#rename column names to Recency, Frequency and Monetary
rfm.rename(columns={'NumWebPurchases': 'Frequency',
                    'Marital_Status2': 'Marital_Status',
                    'Income2': 'Income',
                    'Dt_Customer2': 'Dt_Customer', 
                    'MntTotal': 'Monetary'}, inplace=True)
rfm.head()

,ID,Year_Birth,Education,Marital_Status,Income,Dt_Customer,Recency,Frequency,NumWebVisitsMonth,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Monetary
0,1826,1970,Graduation,Divorced,84835.0,2014-06-16,0,4,1,0,0,0,0,0,783
1,1,1961,Graduation,Single,57091.0,2014-06-15,0,7,5,0,1,0,0,0,540
2,10476,1958,Graduation,Married,67267.0,2014-05-13,0,3,2,0,0,0,0,0,219
3,1386,1967,Graduation,Together,32474.0,2014-05-11,0,1,7,0,0,0,0,0,11
4,5371,1989,Graduation,Single,21474.0,2014-04-08,0,3,7,0,0,1,0,0,57


In [40]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 0 to 2239
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID                 2240 non-null   int64         
 1   Year_Birth         2240 non-null   int64         
 2   Education          2240 non-null   object        
 3   Marital_Status     2240 non-null   object        
 4   Income             2240 non-null   float64       
 5   Dt_Customer        2240 non-null   datetime64[ns]
 6   Recency            2240 non-null   int64         
 7   Frequency          2240 non-null   int64         
 8   NumWebVisitsMonth  2240 non-null   int64         
 9   AcceptedCmp1       2240 non-null   int64         
 10  AcceptedCmp2       2240 non-null   int64         
 11  AcceptedCmp3       2240 non-null   int64         
 12  AcceptedCmp4       2240 non-null   int64         
 13  AcceptedCmp5       2240 non-null   int64         
 14  Monetary

In [41]:
#caegorize 'Year_Birth' as Generation
rfm.loc[rfm['Year_Birth']<=1945, 'Generation'] = 'Silent Gen'
rfm.loc[rfm['Year_Birth'].between(1946,1964), 'Generation'] = 'Baby Boomers'
rfm.loc[rfm['Year_Birth'].between(1965,1980), 'Generation'] = 'Gen X'
rfm.loc[rfm['Year_Birth'].between(1981,1996), 'Generation'] = 'Millennials'
rfm.loc[rfm['Year_Birth']>1997, 'Generation'] = 'Gen Z'

# shift column 'Generation'
column1 = rfm.pop('Generation')
  
# insert column using insert function
rfm.insert(2, 'Generation', column1)

rfm.head()

,ID,Year_Birth,Generation,Education,Marital_Status,Income,Dt_Customer,Recency,Frequency,NumWebVisitsMonth,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Monetary
0,1826,1970,Gen X,Graduation,Divorced,84835.0,2014-06-16,0,4,1,0,0,0,0,0,783
1,1,1961,Baby Boomers,Graduation,Single,57091.0,2014-06-15,0,7,5,0,1,0,0,0,540
2,10476,1958,Baby Boomers,Graduation,Married,67267.0,2014-05-13,0,3,2,0,0,0,0,0,219
3,1386,1967,Gen X,Graduation,Together,32474.0,2014-05-11,0,1,7,0,0,0,0,0,11
4,5371,1989,Millennials,Graduation,Single,21474.0,2014-04-08,0,3,7,0,0,1,0,0,57


In [42]:
#divide 'Income' column as three categories: low, high, & medium
rfm['Income_Category'] = pd.qcut(rfm['Income'], 3, ['low','middle','high'])

# shift column 'Income_Category'
column2 = rfm.pop('Income_Category')
  
# insert column using insert function
rfm.insert(5, 'Income_Category', column2)
rfm.head()

,ID,Year_Birth,Generation,Education,Marital_Status,Income_Category,Income,Dt_Customer,Recency,Frequency,NumWebVisitsMonth,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Monetary
0,1826,1970,Gen X,Graduation,Divorced,high,84835.0,2014-06-16,0,4,1,0,0,0,0,0,783
1,1,1961,Baby Boomers,Graduation,Single,middle,57091.0,2014-06-15,0,7,5,0,1,0,0,0,540
2,10476,1958,Baby Boomers,Graduation,Married,high,67267.0,2014-05-13,0,3,2,0,0,0,0,0,219
3,1386,1967,Gen X,Graduation,Together,low,32474.0,2014-05-11,0,1,7,0,0,0,0,0,11
4,5371,1989,Millennials,Graduation,Single,low,21474.0,2014-04-08,0,3,7,0,0,1,0,0,57


In [43]:
#Calculating RFM Scores, quantile-based discretization function
rfm["R"] = pd.qcut(rfm["Recency"],5,labels=[5,4,3,2,1])
rfm["F"] = pd.qcut(rfm["Frequency"],5,labels=[1,2,3,4,5])
rfm["M"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm["RFM_Score"] = rfm["R"].astype(str) +rfm["F"].astype(str) + rfm["M"].astype(str)
rfm.head()

,ID,Year_Birth,Generation,Education,Marital_Status,Income_Category,Income,Dt_Customer,Recency,Frequency,...,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Monetary,R,F,M,RFM_Score
0,1826,1970,Gen X,Graduation,Divorced,high,84835.0,2014-06-16,0,4,...,0,0,0,0,0,783,5,3,4,534
1,1,1961,Baby Boomers,Graduation,Single,middle,57091.0,2014-06-15,0,7,...,0,1,0,0,0,540,5,5,4,554
2,10476,1958,Baby Boomers,Graduation,Married,high,67267.0,2014-05-13,0,3,...,0,0,0,0,0,219,5,2,3,523
3,1386,1967,Gen X,Graduation,Together,low,32474.0,2014-05-11,0,1,...,0,0,0,0,0,11,5,1,1,511
4,5371,1989,Millennials,Graduation,Single,low,21474.0,2014-04-08,0,3,...,0,0,1,0,0,57,5,2,2,522


In [44]:
#customer segmentation by Recency and Frequency
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [45]:
#create segmentation column
rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,ID,Year_Birth,Generation,Education,Marital_Status,Income_Category,Income,Dt_Customer,Recency,Frequency,...,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Monetary,R,F,M,RFM_Score,Segment
0,1826,1970,Gen X,Graduation,Divorced,high,84835.0,2014-06-16,0,4,...,0,0,0,0,783,5,3,4,534,Potential Loyalists
1,1,1961,Baby Boomers,Graduation,Single,middle,57091.0,2014-06-15,0,7,...,1,0,0,0,540,5,5,4,554,Champions
2,10476,1958,Baby Boomers,Graduation,Married,high,67267.0,2014-05-13,0,3,...,0,0,0,0,219,5,2,3,523,Potential Loyalists
3,1386,1967,Gen X,Graduation,Together,low,32474.0,2014-05-11,0,1,...,0,0,0,0,11,5,1,1,511,New Customers
4,5371,1989,Millennials,Graduation,Single,low,21474.0,2014-04-08,0,3,...,0,1,0,0,57,5,2,2,522,Potential Loyalists


In [46]:
#download as CSV File
from google.colab import files

rfm.to_csv('marketing_clean.csv', encoding = 'utf-8-sig') 
files.download('marketing_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>